In [167]:
import json
import os
import glob

'''
    My Hyporthesis:
    1. tasks 0,1,2,4,6,7,8 the resposnive deisgn will have faster total_times (group B)
    2. task 3,5 nonresponsive design will have a faster time (group A)
        - last article task, last calc task

    Reasons:
    1. Responsive design shows less content making it easier to find and faster to complete task
    2. The participant already knows there that content is from seeing it in earlier tasks and therefore will be able to find it quicker. There is no display difference in these
        two tasks so the time difference could be neglible. However, i think the knowledge of the application gained in previous tasks will result in a smaller total_time for the nonresponsive (group A) participants
'''


'\n    My Hyporthesis:\n    1. tasks 0,1,2,4,6,7,8 the resposnive deisgn will have faster total_times (group B)\n    2. task 3,5 nonresponsive design will have a faster time (group A)\n        - last article task, last calc task\n\n    Reasons:\n    1. Responsive design shows less content making it easier to find and faster to complete task\n    2. The participant already knows there that content is from seeing it in earlier tasks and therefore will be able to find it quicker. There is no display difference in these\n        two tasks so the time difference could be neglible. However, i think the knowledge of the application gained in previous tasks will result in a smaller total_time for the nonresponsive (group A) participants\n'

In [168]:
path = "ExperimentData"
dir_list = os.listdir(path)
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

exp_paths = []

for filename in dir_list:
    if(filename == "34"):
        exp_paths.append(path + "//" + filename + "//ResponsiveAR//Experiment//")

print(exp_paths)

Files and directories in ' ExperimentData ' :
['32', '34', '33', '.DS_Store', '1652652890956', '31', '1652653057225', '30']
['ExperimentData//34//ResponsiveAR//Experiment//']


In [169]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return float(fragments[1])

In [170]:
import pandas as pd 
import numpy as np
import itertools
pd.options.display.max_rows = None

In [171]:
responsiveData = []
experimentEvents = []

def parseFrame(frames):
    frames_list = []
    for frame in frames:
        frameData = {
            "frameNum": int,
            "unixTime": float,
            "timestamp": float,
            "hPos": {"x":float,"y":float,"z":float},
            "hDir": {"x":float,"y":float,"z":float},
            "hRot": {"i":float,"j":float,"k":float},
            "hAngl": {"x":float,"y":float,"z":float},
            "gazeOrigin": {"x":float,"y":float,"z":float},
            "gazeDirection": {"x":float,"y":float,"z":float},
            "rightHandRay": {"x":float,"y":float,"z":float},
            "leftHandRay": {"x":float,"y":float,"z":float},
            "experimentEvents": [],
            "responsiveData": []
        }
        responsiveEvent = frame["responsiveData"]
        experimentEvent = frame["experimentEvents"]
        frameData["frameNum"] = frame["frameNum"]
        frameData["unixTime"] = frame["unixTime"]
        frameData["timestamp"] = frame["timestamp"]
        frameData["hPos"] = frame["hPos"]
        frameData["hDir"] = frame["hDir"]
        frameData["hRot"] = frame["hRot"]
        frameData["hAngl"] = frame["hAngl"]
        frameData["gazeOrigin"] = frame["gazeOrigin"]
        frameData["gazeDirection"] = frame["gazeDirection"]
        frameData["rightHandRay"] = frame["rightHandRay"]
        frameData["leftHandRay"] = frame["leftHandRay"]
        frameData["experimentEvents"] = experimentEvent
        frameData["responsiveData"] = responsiveEvent
        frames_list.append(frameData)
        if responsiveEvent:
            responsiveData.append(responsiveEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)
    return frames_list

def parseSession(session):
    sessionData = {
    "numFrames": int,
    "task": str,
    "start_time": float,
    "end_time": float,
    "total_time": float,
    "sessionNumber": int,
    "isResponsive": int,
    "frames": []
    }
    sessionData["numFrames"] = session["numFrames"]
    sessionData["task"] = session["task"]
    sessionData["start_time"] = session["start_time"]
    sessionData["end_time"] = session["end_time"]
    sessionData["total_time"] = session["total_time"]
    sessionData["sessionNumber"] = session["sessionNumber"]
    sessionData["isResponsive"] = session["isResponsive"]
    return sessionData


In [172]:
data = []
frames = []
allsession = []

for exp in exp_paths:
    experiment_data = {
    "path" : str,
    "sessions": {},
    }
    sessions = {}
    experiment = os.listdir(exp)
    current_path = exp
    experiment_data["path"] = current_path
    experiment.sort(key=parseFileNumber)
    for filename in experiment:
        if filename.endswith(".json"):
            # Prints only text file present in My Folder
            print(filename)
            firstFileFlag = False
            if("Session_0" in filename):
                firstFileFlag = True
            with open(current_path + filename, 'r') as f:
                json_data = json.load(f)
                if(firstFileFlag):
                    session = parseSession(json_data["obj"]["sessionRecordings"][0])
                    num = session["sessionNumber"]
                    frame = parseFrame(json_data["obj"]["sessionRecordings"][0]["frames"])
                    session["frames"] = frame
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
                else:
                    session = parseSession(json_data)
                    num = session["sessionNumber"]
                    frame = parseFrame(json_data["frames"])
                    session["frames"] = frame
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
    experiment_data["sessions"] = sessions
    data.append(experiment_data)

#print(data)


ExperimentSession_0_1652747508436.json
ExperimentSession_1_1652747508436.json
ExperimentSession_2_1652747508436.json
ExperimentSession_3_1652747508436.json
ExperimentSession_4_1652747508436.json
ExperimentSession_5_1652747508436.json
ExperimentSession_6_1652747508436.json
ExperimentSession_7_1652747508436.json
ExperimentSession_8_1652747508436.json
ExperimentSession_9_1652747508436.json


In [173]:
responsive_data = []
nonresponsive_data = []

for d in data:
    #change to Responsive 
    if(d["sessions"][0]["isResponsive"]):
        responsive_data.append(d)
    else:
        nonresponsive_data.append(d)

#print(responsive_data)
#print(nonresponsive_data)

In [174]:
frames
framesUnpacked = list(itertools.chain(*frames))
framesDf = pd.DataFrame(framesUnpacked)

framesDf = framesDf.sort_values(by='timestamp')
framesDf.head(1)

,frameNum,unixTime,timestamp,hPos,hDir,hRot,hAngl,gazeOrigin,gazeDirection,rightHandRay,leftHandRay,experimentEvents,responsiveData
0,1,1652747519357,10921,"{'x': 0.06302555650472641, 'y': -0.00819837581...","{'x': -0.0913006067276001, 'y': -0.23354959487...","{'x': 0.11844566464424133, 'y': -0.04411512613...","{'x': 13.506141662597656, 'y': 354.61215209960...","{'x': 0.06314645707607269, 'y': -0.00843588355...","{'x': -0.08399702608585358, 'y': -0.2360355556...","{'x': 0.0, 'y': 0.0, 'z': 0.0}","{'x': 0.09026230871677399, 'y': 0.931128621101...","[{'unixTime': 1652747519266, 'systemTime': '17...","{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."


In [175]:
leftRay = framesDf.loc[framesDf["leftHandRay"] != {'x': 0.0, 'y': 0.0, 'z': 0.0}]
rightRay = framesDf.loc[framesDf["rightHandRay"] != {'x': 0.0, 'y': 0.0, 'z': 0.0}]
leftRay["leftHandRay"].count()
assert(leftRay["leftHandRay"].count() + rightRay["rightHandRay"].count() > 0)

In [176]:
frame = framesDf.loc[framesDf["frameNum"] >= 59]
frame.head(2)

,frameNum,unixTime,timestamp,hPos,hDir,hRot,hAngl,gazeOrigin,gazeDirection,rightHandRay,leftHandRay,experimentEvents,responsiveData
58,59,1652747525167,16731,"{'x': -0.07504869997501373, 'y': -0.0016013303...","{'x': 0.21886204183101654, 'y': -0.13797138631...","{'x': 0.06218457967042923, 'y': 0.114704258739...","{'x': 7.930476188659668, 'y': 12.7663288116455...","{'x': -0.07476513087749481, 'y': -0.0015327371...","{'x': 0.21690957248210907, 'y': -0.13761293888...","{'x': 2.2704999446868896, 'y': -0.109751254320...","{'x': 0.0, 'y': 0.0, 'z': 0.0}",[],"{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."
59,60,1652747525267,16831,"{'x': -0.07733453810214996, 'y': -0.0023269415...","{'x': 0.2357357293367386, 'y': -0.140738904476...","{'x': 0.06258480995893478, 'y': 0.123681075870...","{'x': 8.090605735778809, 'y': 13.7747611999511...","{'x': -0.07711536437273026, 'y': -0.0022591734...","{'x': 0.23405258357524872, 'y': -0.14043752849...","{'x': 2.040074586868286, 'y': 0.28511965274810...","{'x': 0.0, 'y': 0.0, 'z': 0.0}",[],"{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."


In [177]:
event = framesDf["responsiveData"]
responsiveEventUnpacked = list(itertools.chain(event))
responsiveEventDf = pd.DataFrame(responsiveEventUnpacked)
responsiveEventDf["frameNum"] = framesDf["frameNum"]
#responsiveEventDf

In [178]:
event = framesDf.head(1)["responsiveData"]
eventDf = pd.DataFrame(*event)
eventDf

,name,scale,position,ratio,distance
x,,0.0,0.0,0.0,0.0
y,,0.0,0.0,0.0,0.0
z,,0.0,0.0,0.0,0.0


In [179]:
experimentEvents

experimentEventsUnpacked = list(itertools.chain(*experimentEvents))
experimentEventsDf = pd.DataFrame(experimentEventsUnpacked)
experimentEventsDf = experimentEventsDf.sort_values(by='unixTime')
#experimentEventsDf


In [180]:
#Count number of start events
experimentEventsDf
startEvents = experimentEventsDf[experimentEventsDf['eventName'] == 'start']
#startEvents
assert(startEvents["unixTime"].count() == 9)

In [181]:
#Count number of complete events
completeEvents = experimentEventsDf[experimentEventsDf['eventName'] == 'complete']
assert(completeEvents["unixTime"].count() == 9)

In [182]:
#Count number of instruction events
instructionEvents = experimentEventsDf[experimentEventsDf['eventName'] == 'instruction']
assert(instructionEvents["unixTime"].count() == 9)

In [183]:
#Count number of answer events
answerEvents = experimentEventsDf[experimentEventsDf['eventName'] == 'answer']
assert(answerEvents["unixTime"].count() >= 9)

In [184]:
totalTime = completeEvents["unixTime"].to_numpy() - startEvents["unixTime"].to_numpy()
totalTime

for time in totalTime:
    assert(time > 0)

In [185]:
responsiveData
responsiveDataUnpacked = list(itertools.chain(responsiveData))
responsiveDataDf = pd.DataFrame(responsiveDataUnpacked)
#responsiveDataDf = responsiveDataDf.sort_values(by='sessionNumber')
#responsiveDataDf

In [186]:
allsession

sessionsUnpacked = list(itertools.chain(allsession))
sessionsDf = pd.DataFrame(sessionsUnpacked)
sessionsDf = sessionsDf.sort_values(by='sessionNumber')

sessionsDf = sessionsDf.loc[sessionsDf["sessionNumber"] < 9 ]
sessionsDf


,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
0,210,The task is to click on the 2nd article title....,1652747526667,1652747540607,13940,0,False,"[{'frameNum': 1, 'unixTime': 1652747519357, 't..."
1,255,The task is to click the 5th article title.\n\...,1652747547557,1652747566848,19291,1,False,"[{'frameNum': 211, 'unixTime': 1652747540698, ..."
2,127,The task is to click the Author of the 4th art...,1652747575188,1652747579718,4530,2,False,"[{'frameNum': 466, 'unixTime': 1652747566897, ..."
3,191,The task is to click the Date of the 1st artic...,1652747586188,1652747599098,12910,3,False,"[{'frameNum': 593, 'unixTime': 1652747579758, ..."
4,485,The task is to type the phone number 949-927-4...,1652747615448,1652747648919,33471,4,False,"[{'frameNum': 784, 'unixTime': 1652747599148, ..."
5,361,The task is to type phone number 805-UCSB-EDU ...,1652747659553,1652747685749,26196,5,False,"[{'frameNum': 1269, 'unixTime': 1652747649039,..."
6,389,The task is to find the current temperature on...,1652747702301,1652747727120,24819,6,False,"[{'frameNum': 1630, 'unixTime': 1652747685829,..."
7,606,The task is to find the current location on th...,1652747740940,1652747791451,50511,7,False,"[{'frameNum': 2019, 'unixTime': 1652747727289,..."
8,1464,The task is to find the low temperature on th...,1652747802011,1652747952843,150832,8,False,"[{'frameNum': 2625, 'unixTime': 1652747791601,..."


In [187]:
avg = 0
count = 0
for time in sessionsDf["total_time"]:
    assert(time > 0)
    avg = time + avg
    count = count + 1

print("avg: " + str(avg/count) + " milliseconds")

avg: 37388.88888888889 milliseconds


In [188]:
for i in range(0,9):
    avg_ratio = 0
    sessionST = startEvents.loc[startEvents["task_number"] == i]
    sessionST = sessionST["unixTime"].to_numpy()[0]

    sessionET = completeEvents.loc[completeEvents["task_number"] == i]
    sessionET = sessionET["unixTime"].to_numpy()[0]

    taskDF = framesDf.loc[framesDf["unixTime"] >= sessionST]
    taskDF = taskDF.loc[taskDF["unixTime"] <= sessionET]
    
    #task2DF
    print("number of frames in task " + str(i) + " is " + str(taskDF.shape[0]))

number of frames in task 0 is 136
number of frames in task 1 is 186
number of frames in task 2 is 44
number of frames in task 3 is 126
number of frames in task 4 is 322
number of frames in task 5 is 255
number of frames in task 6 is 224
number of frames in task 7 is 470
number of frames in task 8 is 1359


In [189]:
for i in range(0,9):
    sessionST = instructionEvents.loc[instructionEvents["task_number"] == i]
    sessionST = sessionST["unixTime"].to_numpy()[0]

    sessionET = startEvents.loc[startEvents["task_number"] == i]
    sessionET = sessionET["unixTime"].to_numpy()[0]

    taskDF = framesDf.loc[framesDf["unixTime"] >= sessionST]
    taskDF = taskDF.loc[taskDF["unixTime"] <= sessionET]
    #task2DF
    print("number of frames for task instruction " + str(i) + " is " + str(taskDF.shape[0]))

number of frames for task instruction 0 is 74
number of frames for task instruction 1 is 69
number of frames for task instruction 2 is 83
number of frames for task instruction 3 is 65
number of frames for task instruction 4 is 163
number of frames for task instruction 5 is 106
number of frames for task instruction 6 is 165
number of frames for task instruction 7 is 136
number of frames for task instruction 8 is 105


In [190]:
for i in range(0,9):
    guesses = answerEvents.loc[answerEvents["task_number"] == i]
    print("number of guesses for task " + str(i) + " is " + str(guesses.shape[0]))


number of guesses for task 0 is 1
number of guesses for task 1 is 3
number of guesses for task 2 is 1
number of guesses for task 3 is 4
number of guesses for task 4 is 1
number of guesses for task 5 is 1
number of guesses for task 6 is 8
number of guesses for task 7 is 23
number of guesses for task 8 is 46


In [191]:
responsiveEventDf.head(1)

,name,scale,position,ratio,distance,frameNum
0,,"{'x': 0.0, 'y': 0.0, 'z': 0.0}","{'x': 0.0, 'y': 0.0, 'z': 0.0}",0.0,0.0,1


In [192]:
bool_val = sessionsDf["isResponsive"].to_numpy()[0]
for i in range(0,9):
    assert(sessionsDf["isResponsive"].to_numpy()[i] == bool_val)


In [193]:
if(sessionsDf["isResponsive"].to_numpy()[0]):
    for i in range(0,9):
        avg_ratio = 0
        sessionST = startEvents.loc[startEvents["task_number"] == i]
        sessionST = sessionST["unixTime"].to_numpy()[0]

        sessionET = completeEvents.loc[completeEvents["task_number"] == i]
        sessionET = sessionET["unixTime"].to_numpy()[0]

        taskDF = framesDf.loc[framesDf["unixTime"] >= sessionST]
        taskDF = taskDF.loc[taskDF["unixTime"] <= sessionET]

        event = taskDF["responsiveData"]
        responsiveUnpacked = list(itertools.chain(event))
        responsiveDf = pd.DataFrame(responsiveUnpacked)

        ratio = responsiveDf["ratio"]
        dist = responsiveDf["distance"]

        print("avg dist of object during task " + str(i) + " is " + str(dist.sum() / dist.shape[0]))
        print("avg ratio of object during task " + str(i) + " is " + str(ratio.sum() / ratio.shape[0]))